2025 - Employment Permit by Nationality

In [1]:
year = 2025

# Reading the raw data in the folder on my personal computer
import pandas as pd

# Path to the main and aux files
path_main = f"G:/My Drive/ESTUDOS DATA SCIENCE/ie-employment-permit/data/raw_data/{year}/permits-by-nationality-{year}.xlsx"
path_aux = "G:/My Drive/ESTUDOS DATA SCIENCE/ie-employment-permit/data/world_countries.xlsx"

# Reads the main file, skipping unnecessary rows and setting the header
df = pd.read_excel(
    path_main,
    skiprows=[2],
    header=1
)

# Clean column names (very common on this project)
df.columns = df.columns.str.strip()

In [2]:
# I had to replace "Nationality" by just "Country" once that this analysis is about countries
df.rename(columns={"Nationality": "Country"}, inplace=True)

# Reads the aux file which has the mapping of countries to continents and regions
aux = pd.read_excel(path_aux, sheet_name="Aux_Table")

# Creating the mapping dictionary
mapping = dict(zip(aux["Before"], aux["After"]))

# Replacing the countries names based on the mapping dictionary
df["Country"] = df["Country"].replace(mapping)

# Creating a new column because of the new structure
df["Year"] = str(year)

# Sorting the year column to be the first one 
df = df[["Year"] + [col for col in df.columns if col != "Year"]]

In [3]:
# I grouped by country because in some cases when I was fixing I replaced the old name to the new name, ie Hong Kong -> China making double China
df_grouped = df.groupby(["Year", "Country"], dropna=False).agg({
    "Issued": "sum",
    "Refused": "sum",
    "Withdrawn": "sum"
}).reset_index()

# Filter rows where at least one of the numeric columns is non-zero
numeric_cols = ["Issued", "Refused", "Withdrawn"]
df_grouped = df_grouped[df_grouped[numeric_cols].any(axis=1)]

In [4]:
# I used the extension .csv because is lighter and easy to work with some libraries like pandas, sqlalchemy
df_grouped.to_csv(f"G:/My Drive/ESTUDOS DATA SCIENCE/ie-employment-permit/data/{year}/permits-by-nationality-{year}.csv", index=False)

print(df_grouped)

     Year      Country  Issued  Refused  Withdrawn
0    2025  Afghanistan     4.0      3.0        5.0
1    2025      Albania    12.0      4.0        7.0
2    2025      Algeria    21.0      5.0       38.0
4    2025       Angola     0.0      0.0        2.0
7    2025    Argentina    70.0     14.0       28.0
..    ...          ...     ...      ...        ...
176  2025    Venezuela    14.0      2.0        6.0
177  2025      Vietnam    84.0      4.0       46.0
178  2025        Yemen     3.0      0.0        6.0
179  2025       Zambia    14.0      3.0       13.0
180  2025     Zimbabwe   607.0     84.0      614.0

[126 rows x 5 columns]
